In [2]:
import pandas as pd

df = pd.read_csv("fruit_veg_fresh_stale.csv")

df_fruit = df[df['type'] == 'fruit'].copy()

print(df_fruit['label'].value_counts())


label
Stale Orange          4411
Banana Ripe           3687
Stale Banana          2903
Stale Apple           2787
Fresh Banana          2581
Fresh Orange          2466
Grape Unripe          2183
Stale Grape           1866
Banana Unripe         1859
Fresh Apple           1693
Grape Ripe            1579
Stale Mango           1293
Melon Ripe            1140
Fresh Mango           1000
Fresh Lulo            1000
Fresh Lemon           1000
Stale Lulo            1000
Stale Lemon           1000
Fresh Tamarillo       1000
Fresh Strawberry      1000
Stale Strawberry      1000
Stale Tamarillo       1000
Apple Ripe             922
Apple Overripe         563
Apple Unripe           476
Banana Overripe        462
Mango Ripe             440
Mango Unripe           433
Mango Overripe         400
Peach Unripe           360
Grape Overripe         358
Stale Bitter Gourd     357
Fresh Bitter Gourd     327
Stale Pear             227
Orange Ripe            226
Peach Ripe             195
Stale Peach           

In [3]:
import re

valid_qualities = ['ripe', 'unripe', 'overripe', 'stale', 'fresh']

def extract_quality(label):
    words = re.findall(r'\b\w+\b', label.lower())
    for q in valid_qualities:
        if q in words:
            return q
    return 'unknown'

df_fruit['label'] = df_fruit['label'].apply(extract_quality)

df_fruit = df_fruit[df_fruit['label'] != 'unknown']


In [4]:
df_fruit['label'].value_counts()

label
stale       18060
fresh       12067
ripe         8302
unripe       5745
overripe     2196
Name: count, dtype: int64

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle

le = LabelEncoder()
train_df, val_df = train_test_split(df_fruit, test_size=0.2, stratify=df_fruit['label'], random_state=42)

train_df['label_enc'] = le.fit_transform(train_df['label'])
val_df['label_enc'] = le.transform(val_df['label']) 

with open("fruit_label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label_enc']),
    y=train_df['label_enc']
)
class_weights = dict(zip(np.unique(train_df['label_enc']), class_weights))
print("Class weights:", class_weights)

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
out = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=out)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_cb = ModelCheckpoint(
    "fruit_quality_checkpoint.h5",
    save_weights_only=False,
    save_best_only=False,
    save_freq='epoch',
    verbose=1
)

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[checkpoint_cb],
    class_weight=class_weights
)

Class weights: {0: 0.7685900756241583, 1: 4.222652248150256, 2: 1.1170129479072568, 3: 0.5135105204872646, 4: 1.6142732811140121}
Found 37096 validated image filenames belonging to 5 classes.
Found 9274 validated image filenames belonging to 5 classes.
Epoch 1/10
1160/1160 [==============================] - ETA: 0s - loss: 0.7108 - accuracy: 0.7385
Epoch 1: saving model to fruit_quality_checkpoint.h5


c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1160/1160 [==============================] - 837s 718ms/step - loss: 0.7108 - accuracy: 0.7385 - val_loss: 0.4806 - val_accuracy: 0.8297
Epoch 2/10
  61/1160 [>.............................] - ETA: 10:18 - loss: 0.4864 - accuracy: 0.8263

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("fruit_quality_checkpoint.h5")

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,               
    initial_epoch=3,        
    class_weight=class_weights,
    callbacks=[checkpoint_cb]
)

NameError: name 'train_df' is not defined

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle

le = LabelEncoder()
train_df, val_df = train_test_split(df_fruit, test_size=0.2, stratify=df_fruit['label'], random_state=42)

train_df['label_enc'] = le.fit_transform(train_df['label'])
val_df['label_enc'] = le.transform(val_df['label']) 

with open("fruit_label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label_enc']),
    y=train_df['label_enc']
)
class_weights = dict(zip(np.unique(train_df['label_enc']), class_weights))
print("Class weights:", class_weights)

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

checkpoint_cb = ModelCheckpoint(
    "fruit_quality_checkpoint.h5",
    save_weights_only=False,
    save_best_only=False,
    save_freq='epoch',
    verbose=1
)
from tensorflow.keras.models import load_model

model = load_model("final_model.h5")

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,            
    initial_epoch=9,     
    class_weight=class_weights,
    callbacks=[checkpoint_cb]
)

Class weights: {0: 0.7685900756241583, 1: 4.222652248150256, 2: 1.1170129479072568, 3: 0.5135105204872646, 4: 1.6142732811140121}
Found 37096 validated image filenames belonging to 5 classes.
Found 9274 validated image filenames belonging to 5 classes.
Epoch 10/10
1160/1160 [==============================] - ETA: 0s - loss: 0.1804 - accuracy: 0.9310
Epoch 10: saving model to fruit_quality_checkpoint.h5


c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1160/1160 [==============================] - 803s 690ms/step - loss: 0.1804 - accuracy: 0.9310 - val_loss: 0.1973 - val_accuracy: 0.9281


In [ ]:
model = load_model("final_model.h5")

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,            
    initial_epoch=5,     
    class_weight=class_weights,
    callbacks=[checkpoint_cb]
)

Epoch 6/10
1160/1160 [==============================] - ETA: 0s - loss: 0.2399 - accuracy: 0.9080
Epoch 6: saving model to fruit_quality_checkpoint.h5
1160/1160 [==============================] - 828s 711ms/step - loss: 0.2399 - accuracy: 0.9080 - val_loss: 0.2792 - val_accuracy: 0.8920
Epoch 7/10
1160/1160 [==============================] - ETA: 0s - loss: 0.2210 - accuracy: 0.9154
Epoch 7: saving model to fruit_quality_checkpoint.h5
1160/1160 [==============================] - 801s 691ms/step - loss: 0.2210 - accuracy: 0.9154 - val_loss: 0.2622 - val_accuracy: 0.9014
Epoch 8/10
1160/1160 [==============================] - ETA: 0s - loss: 0.2052 - accuracy: 0.9224
Epoch 8: saving model to fruit_quality_checkpoint.h5
1160/1160 [==============================] - 799s 689ms/step - loss: 0.2052 - accuracy: 0.9224 - val_loss: 0.2338 - val_accuracy: 0.9133
Epoch 9/10
1160/1160 [==============================] - ETA: 0s - loss: 0.1920 - accuracy: 0.9281
Epoch 9: saving model to fruit_quality

KeyboardInterrupt: 

In [13]:

model.save("fruit_quality_model.keras")

In [7]:
model = load_model("fruit_quality_checkpoint.h5")
model.save("final_model.keras")
model.save("final_model.h5")


c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
